In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [13]:
from sklearn.preprocessing import OneHotEncoder

In [42]:
class GaussianDNN(nn.Module):
    def __init__(self, num_feats: int):
        super(GaussianDNN, self).__init__()
        self.fc1 = nn.Linear(num_feats, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.mean_output = nn.Linear(256, 1)
        self.logvar_output = nn.Linear(256, 1)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        mean = self.mean_output(x)
        logvar = self.logvar_output(x)
        return mean, logvar

In [43]:
def train_gaussian_dnn(train_loader, model, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        mean, logvar = model(inputs)
        
        loss = 0.5 * (torch.exp(-logvar) * (targets - mean)**2 + logvar).mean()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def evaluate_gaussian_dnn(val_loader, model):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            mean, logvar = model(inputs)
            
            loss = 0.5 * (torch.exp(-logvar) * (targets - mean)**2 + logvar).mean()
            running_loss += loss.item() * inputs.size(0)
    return running_loss / len(val_loader.dataset)

In [44]:
dta = pd.read_csv("../data/fall-msd-treatment.csv")

In [45]:
dta = dta[dta['payoff'] > 0.0]

In [67]:
day_feats = pd.to_datetime(dta['last_scanned_datetime']).dt.day_of_week.values.reshape(-1, 1)

In [46]:
one_hot_prod = OneHotEncoder(sparse=False)

In [47]:
one_hot_store = OneHotEncoder(sparse=False)

In [63]:
prod_feats = one_hot_prod.fit_transform(dta['product_id'].values.reshape(-1, 1))

In [49]:
one_hot_prod.fit_transform(dta['store_id'].values.reshape(-1, 1))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [68]:
day_feats

array([[4],
       [4],
       [4],
       ...,
       [2],
       [2],
       [2]])

In [65]:
prod_feats.shape

(7303, 133)

In [70]:
x_values = np.concatenate([prod_feats, day_feats], axis=-1)

In [71]:
y_values = dta['payoff'].values

In [76]:
# Convert data to PyTorch tensors
x_tensor = torch.tensor(x_values, dtype=torch.float32).unsqueeze(1)
y_tensor = torch.tensor(y_values, dtype=torch.float32).unsqueeze(1)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, test_size=0.1, random_state=42)

# Create a DataLoader for training data
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [77]:
device = torch.device("cpu")

In [78]:
# Instantiate and train the Gaussian DNN model
gaussian_dnn_model = GaussianDNN(num_feats =  x_values.shape[1]).to(device)
optimizer = optim.Adam(gaussian_dnn_model.parameters(), lr=0.01)

In [79]:
num_epochs = 20
progress_bar = tqdm(range(num_epochs), desc="Training Gaussian DNN", unit="epoch")
for epoch in progress_bar:
    train_loss = train_gaussian_dnn(train_loader, gaussian_dnn_model, optimizer)
    val_loss = evaluate_gaussian_dnn(val_loader, gaussian_dnn_model)
    
    progress_bar.set_postfix({"Train Loss": f"{train_loss:.4f}", "Val Loss": f"{val_loss:.4f}"})

Training Gaussian DNN: 100%|█| 20/20 [00:11<00:00,  1.67epoch/s, Train Loss=1.4750, Val Lo
